# EKE

This notebook takes raw, 5-daily geostrophic surface velocity data from the `OCCIPUT` simulations (provided by Sally Close, calculated from detrended SLA) and estimates `EKE(x,y,t)`. It then resamples down to monthly data and saves the output locally for further analysis.

**A few notes on this process:**
 * I have saved individual EKE files to make the processing faster (avoiding memory limitations on my laptop), at a cost of slight additional disk space. 
 * To offset this, EKE files are re-sampled onto a monthly timeseries, which i think is fine for our purposes.
 * There are some crazy errors in the geostrophic velocity files. Under advice from Sally I have just eliminated  any EKE values  exceeding 2 m^2/s^2.
 * Processing seemed to all run smoothly, as far as I can tell.
 
**AH - 11 April 2019**

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import IPython.display
import pandas as pd
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

In [3]:
HardDisk = '/Volumes/Lacie/'
EnsembleDir = 'geouv/'

In [4]:
## Only need to load grid once
gridfilename = HardDisk+'ORCA025.L75-MJM101.1_mesh_hgr.nc'
gridds = xr.open_dataset(gridfilename)
lon = gridds.nav_lon.isel(y=0).values.copy()
lon[:430] = lon[:430]-360
lat = gridds.nav_lat.isel(x=387).values

In [61]:
%%time
for ii in range(1,51):
    for year in range(1979,2016):
        Year = '%d' % year
        i00 = '%03d' % ii
        print('Now doing '+i00+' - '+ Year)

        Member = 'e'+i00
        ufilename = HardDisk+EnsembleDir+Member+'/ugeo_'+i00+'_'+Year+'.nc'
        vfilename = HardDisk+EnsembleDir+Member+'/ugeo_'+i00+'_'+Year+'.nc'
        ekefilename = HardDisk+EnsembleDir+Member+'/eke_'+i00+'_'+Year+'.nc'

        uds = xr.open_dataset(ufilename)
        vds = xr.open_dataset(vfilename)

        u=uds.vozocrtx.values
        u2 = np.empty([73,1021,1442])
        u2[:,1:,:] = 0.5*(u[:,:-1,:]+u[:,1:,:])
        u2[:,0,:] = u[:,0,:]
        u2 = u2*u2
    
        v=vds.vozocrtx.values
        v2 = np.empty([73,1021,1442])
        v2[:,:,1:] = 0.5*(v[:,:,:-1]+v[:,:,1:])
        v2[:,:,0] = 0.5*(v[:,:,-1]+v[:,:,0])
        v2 = v2*v2

        eke = 0.5*(u2+v2)
        # FUDGE - reset funny values to zero
        eke[eke>2] = 0
        ekeda  = xr.DataArray(eke[:,:-1,1:], coords=[('time', uds.time_counter.values), 
                                            ('lat', lat[:-1]),
                                            ('lon', lon[1:])], name='EKE',)

        eke_monthly = ekeda.resample(time='M').mean('time')    
        eke_monthly.to_netcdf(ekefilename)

Now doing 001 - 1979
Now doing 001 - 1980
Now doing 001 - 1981
Now doing 001 - 1982
Now doing 001 - 1983
Now doing 001 - 1984
Now doing 001 - 1985
Now doing 001 - 1986
Now doing 001 - 1987
Now doing 001 - 1988
Now doing 001 - 1989
Now doing 001 - 1990
Now doing 001 - 1991
Now doing 001 - 1992
Now doing 001 - 1993
Now doing 001 - 1994
Now doing 001 - 1995
Now doing 001 - 1996
Now doing 001 - 1997
Now doing 001 - 1998
Now doing 001 - 1999
Now doing 001 - 2000
Now doing 001 - 2001
Now doing 001 - 2002
Now doing 001 - 2003
Now doing 001 - 2004
Now doing 001 - 2005
Now doing 001 - 2006
Now doing 001 - 2007
Now doing 001 - 2008
Now doing 001 - 2009
Now doing 001 - 2010
Now doing 001 - 2011
Now doing 001 - 2012
Now doing 001 - 2013
Now doing 001 - 2014
Now doing 001 - 2015
Now doing 002 - 1979
Now doing 002 - 1980
Now doing 002 - 1981
Now doing 002 - 1982
Now doing 002 - 1983
Now doing 002 - 1984
Now doing 002 - 1985
Now doing 002 - 1986
Now doing 002 - 1987
Now doing 002 - 1988
Now doing 002

Now doing 011 - 2000
Now doing 011 - 2001
Now doing 011 - 2002
Now doing 011 - 2003
Now doing 011 - 2004
Now doing 011 - 2005
Now doing 011 - 2006
Now doing 011 - 2007
Now doing 011 - 2008
Now doing 011 - 2009
Now doing 011 - 2010
Now doing 011 - 2011
Now doing 011 - 2012
Now doing 011 - 2013
Now doing 011 - 2014
Now doing 011 - 2015
Now doing 012 - 1979
Now doing 012 - 1980
Now doing 012 - 1981
Now doing 012 - 1982
Now doing 012 - 1983
Now doing 012 - 1984
Now doing 012 - 1985
Now doing 012 - 1986
Now doing 012 - 1987
Now doing 012 - 1988
Now doing 012 - 1989
Now doing 012 - 1990
Now doing 012 - 1991
Now doing 012 - 1992
Now doing 012 - 1993
Now doing 012 - 1994
Now doing 012 - 1995
Now doing 012 - 1996
Now doing 012 - 1997
Now doing 012 - 1998
Now doing 012 - 1999
Now doing 012 - 2000
Now doing 012 - 2001
Now doing 012 - 2002
Now doing 012 - 2003
Now doing 012 - 2004
Now doing 012 - 2005
Now doing 012 - 2006
Now doing 012 - 2007
Now doing 012 - 2008
Now doing 012 - 2009
Now doing 012

Now doing 022 - 1984
Now doing 022 - 1985
Now doing 022 - 1986
Now doing 022 - 1987
Now doing 022 - 1988
Now doing 022 - 1989
Now doing 022 - 1990
Now doing 022 - 1991
Now doing 022 - 1992
Now doing 022 - 1993
Now doing 022 - 1994
Now doing 022 - 1995
Now doing 022 - 1996
Now doing 022 - 1997
Now doing 022 - 1998
Now doing 022 - 1999
Now doing 022 - 2000
Now doing 022 - 2001
Now doing 022 - 2002
Now doing 022 - 2003
Now doing 022 - 2004
Now doing 022 - 2005
Now doing 022 - 2006
Now doing 022 - 2007
Now doing 022 - 2008
Now doing 022 - 2009
Now doing 022 - 2010
Now doing 022 - 2011
Now doing 022 - 2012
Now doing 022 - 2013
Now doing 022 - 2014
Now doing 022 - 2015
Now doing 023 - 1979
Now doing 023 - 1980
Now doing 023 - 1981
Now doing 023 - 1982
Now doing 023 - 1983
Now doing 023 - 1984
Now doing 023 - 1985
Now doing 023 - 1986
Now doing 023 - 1987
Now doing 023 - 1988
Now doing 023 - 1989
Now doing 023 - 1990
Now doing 023 - 1991
Now doing 023 - 1992
Now doing 023 - 1993
Now doing 023

Now doing 032 - 2005
Now doing 032 - 2006
Now doing 032 - 2007
Now doing 032 - 2008
Now doing 032 - 2009
Now doing 032 - 2010
Now doing 032 - 2011
Now doing 032 - 2012
Now doing 032 - 2013
Now doing 032 - 2014
Now doing 032 - 2015
Now doing 033 - 1979
Now doing 033 - 1980
Now doing 033 - 1981
Now doing 033 - 1982
Now doing 033 - 1983
Now doing 033 - 1984
Now doing 033 - 1985
Now doing 033 - 1986
Now doing 033 - 1987
Now doing 033 - 1988
Now doing 033 - 1989
Now doing 033 - 1990
Now doing 033 - 1991
Now doing 033 - 1992
Now doing 033 - 1993
Now doing 033 - 1994
Now doing 033 - 1995
Now doing 033 - 1996
Now doing 033 - 1997
Now doing 033 - 1998
Now doing 033 - 1999
Now doing 033 - 2000
Now doing 033 - 2001
Now doing 033 - 2002
Now doing 033 - 2003
Now doing 033 - 2004
Now doing 033 - 2005
Now doing 033 - 2006
Now doing 033 - 2007
Now doing 033 - 2008
Now doing 033 - 2009
Now doing 033 - 2010
Now doing 033 - 2011
Now doing 033 - 2012
Now doing 033 - 2013
Now doing 033 - 2014
Now doing 033

Now doing 043 - 1989
Now doing 043 - 1990
Now doing 043 - 1991
Now doing 043 - 1992
Now doing 043 - 1993
Now doing 043 - 1994
Now doing 043 - 1995
Now doing 043 - 1996
Now doing 043 - 1997
Now doing 043 - 1998
Now doing 043 - 1999
Now doing 043 - 2000
Now doing 043 - 2001
Now doing 043 - 2002
Now doing 043 - 2003
Now doing 043 - 2004
Now doing 043 - 2005
Now doing 043 - 2006
Now doing 043 - 2007
Now doing 043 - 2008
Now doing 043 - 2009
Now doing 043 - 2010
Now doing 043 - 2011
Now doing 043 - 2012
Now doing 043 - 2013
Now doing 043 - 2014
Now doing 043 - 2015
Now doing 044 - 1979
Now doing 044 - 1980
Now doing 044 - 1981
Now doing 044 - 1982
Now doing 044 - 1983
Now doing 044 - 1984
Now doing 044 - 1985
Now doing 044 - 1986
Now doing 044 - 1987
Now doing 044 - 1988
Now doing 044 - 1989
Now doing 044 - 1990
Now doing 044 - 1991
Now doing 044 - 1992
Now doing 044 - 1993
Now doing 044 - 1994
Now doing 044 - 1995
Now doing 044 - 1996
Now doing 044 - 1997
Now doing 044 - 1998
Now doing 044

In [16]:
%%time
for year in range(1979,2016):
    Year = '%d' % year
    print('Now doing '+ Year)

    tauxfilename = HardDisk+EnsembleDir+'taux/'+'ORCA025.L75-OCCITENS.001_y'+Year+'.5d_gridU.nc'
    tauyfilename = HardDisk+EnsembleDir+'tauy/'+'ORCA025.L75-OCCITENS.001_y'+Year+'.5d_gridV.nc'
    taufilename = HardDisk+EnsembleDir+'tau/'+'/tau_'+Year+'.nc'

    tauxds = xr.open_dataset(tauxfilename)
    tauyds = xr.open_dataset(tauyfilename)

    # first, on v grid:
    tauy=tauyds.sometauy.values
    tauy2 = np.empty([73,1021,1442])
    tauy2[:,1:,:] = 0.5*(tauy[:,:-1,:]+tauy[:,1:,:])
    tauy2[:,0,:] = tauy[:,0,:]
    tauy2 = tauy2*tauy2
    
    taux=tauxds.sozotaux.values
    taux2 = np.empty([73,1021,1442])
    taux2[:,:,1:] = 0.5*(taux[:,:,:-1]+taux[:,:,1:])
    taux2[:,:,0] = 0.5*(taux[:,:,-1]+taux[:,:,0])
    taux2 = taux2*taux2

    tau = (taux2+tauy2)**0.5
    tauda  = xr.DataArray(tau[:,:-1,1:], coords=[('time', tauxds.time_counter.values), 
                                            ('lat', lat[:-1]),
                                            ('lon', lon[1:])], name='tau',)

    tau_monthly = tauda.resample(time='M').mean('time')    
    tau_monthly.to_netcdf(taufilename)

Now doing 1979
Now doing 1980
Now doing 1981
Now doing 1982
Now doing 1983
Now doing 1984
Now doing 1985
Now doing 1986
Now doing 1987
Now doing 1988
Now doing 1989
Now doing 1990
Now doing 1991
Now doing 1992
Now doing 1993
Now doing 1994
Now doing 1995
Now doing 1996
Now doing 1997
Now doing 1998
Now doing 1999
Now doing 2000
Now doing 2001
Now doing 2002
Now doing 2003
Now doing 2004
Now doing 2005
Now doing 2006
Now doing 2007
Now doing 2008
Now doing 2009
Now doing 2010
Now doing 2011
Now doing 2012
Now doing 2013
Now doing 2014
Now doing 2015
CPU times: user 8min 6s, sys: 2min, total: 10min 7s
Wall time: 14min 40s
